$
% START OF MACRO DEF
% DO NOT EDIT IN INDIVIDUAL NOTEBOOKS, BUT IN macros.py
%
\newcommand{\Reals}{\mathbb{R}}
\newcommand{\Expect}[0]{\mathbb{E}}
\newcommand{\NormDist}{\mathcal{N}}
%
\newcommand{\DynMod}[0]{\mathscr{M}}
\newcommand{\ObsMod}[0]{\mathscr{H}}
%
\newcommand{\mat}[1]{{\mathbf{{#1}}}} 
%\newcommand{\mat}[1]{{\pmb{\mathsf{#1}}}}
\newcommand{\bvec}[1]{{\mathbf{#1}}} 
%
\newcommand{\trsign}{{\mathsf{T}}} 
\newcommand{\tr}{^{\trsign}} 
\newcommand{\tn}[1]{#1} 
\newcommand{\ceq}[0]{\mathrel{≔}}
%
\newcommand{\I}[0]{\mat{I}} 
\newcommand{\K}[0]{\mat{K}}
\newcommand{\bP}[0]{\mat{P}}
\newcommand{\bH}[0]{\mat{H}}
\newcommand{\bF}[0]{\mat{F}}
\newcommand{\R}[0]{\mat{R}}
\newcommand{\Q}[0]{\mat{Q}}
\newcommand{\B}[0]{\mat{B}}
\newcommand{\C}[0]{\mat{C}}
\newcommand{\Ri}[0]{\R^{-1}}
\newcommand{\Bi}[0]{\B^{-1}}
\newcommand{\X}[0]{\mat{X}}
\newcommand{\A}[0]{\mat{A}}
\newcommand{\Y}[0]{\mat{Y}}
\newcommand{\E}[0]{\mat{E}}
\newcommand{\U}[0]{\mat{U}}
\newcommand{\V}[0]{\mat{V}}
%
\newcommand{\x}[0]{\bvec{x}}
\newcommand{\y}[0]{\bvec{y}}
\newcommand{\z}[0]{\bvec{z}}
\newcommand{\q}[0]{\bvec{q}}
\newcommand{\br}[0]{\bvec{r}}
\newcommand{\bb}[0]{\bvec{b}}
%
\newcommand{\bx}[0]{\bvec{\bar{x}}}
\newcommand{\by}[0]{\bvec{\bar{y}}}
\newcommand{\barB}[0]{\mat{\bar{B}}}
\newcommand{\barP}[0]{\mat{\bar{P}}}
\newcommand{\barC}[0]{\mat{\bar{C}}}
\newcommand{\barK}[0]{\mat{\bar{K}}}
%
\newcommand{\D}[0]{\mat{D}}
\newcommand{\Dobs}[0]{\mat{D}_{\text{obs}}}
\newcommand{\Dmod}[0]{\mat{D}_{\text{obs}}}
%
\newcommand{\ones}[0]{\bvec{1}} 
\newcommand{\AN}[0]{\big( \I_N - \ones \ones\tr / N \big)}
%
% END OF MACRO DEF
$

# Data assimilation (DA) & the ensemble Kalman filter (EnKF)
*Copyright (c) 2020, Patrick N. Raanes

### Jupyter notebooks is:
the format used for these tutorials. Notebooks combine **cells** of code (Python) with cells of text (markdown). 
The exercises in these tutorials only require light Python experience.
For example, edit the cell below (double-click it),
insert your name,
and run it (press "Run" in the toolbar).

In [ ]:
name = "Batman"
print("Hello world! I'm " + name)
for i,c in enumerate(name):
    print(i,c)

You will likely be more efficient if you know these **keyboard shortcuts**:

| Navigate                      | Edit              | Exit           | Run                              | Run & go to next                  |
| -------------                 | : ------------- : | -------------  | : --- :                          | : ------------- :                 |
| <kbd>↓</kbd> and <kbd>↑</kbd> | <kbd>Enter</kbd>  | <kbd>Esc</kbd> | <kbd>Ctrl</kbd>+<kbd>Enter</kbd> | <kbd>Shift</kbd>+<kbd>Enter</kbd> |

When you open a notebook it starts a **session (kernel/runtime)** of Python in the background. All of the Python code cells (in a given notebook) are connected (they use the same Python kernel and thus share variables, functions, and classes). Thus, the **order** in which you run the cells matters. For example:

<mark><font size="-1">
    The 1st two code cells in each tutorial will be the following, which <em>you must run before any other</em>:
</font></mark>

In [ ]:
!wget -qO- https://raw.githubusercontent.com/nansencenter/DA-tutorials/master/notebooks/resources/colab_bootstrap.sh | bash -s

In [ ]:
from resources.workspace import *

One thing you must know is how to **restart** the Python session, which clears all of your variables, functions, etc, so that you can start over. Test this now by going through the top menu bar: `Kernel` → `Restart & Clear Output`. But rembember to run the above cell again!

There is a huge amount of libraries available in **Python**, including the popular `scipy` (with `numpy` at its core) and `matplotlib` packages. These were imported (and abbreviated) as `sp`, `np`, and `mpl` and `plt` in the previous cell. Try them out by running the following:

In [ ]:
# Use numpy's arrays for vectors and matrices. Example constructions:
a = np.arange(10) # Alternatively: np.array([0,1,2,3,4,5,6,7,8,9])
I = 2*np.eye(10)  # Alternatively: np.diag(2*np.ones(10))

print("Indexing examples:")
print("a        =", a)
print("a[3]     =", a[3])
print("a[0:3]   =", a[0:3])
print("a[:3]    =", a[:3])
print("a[3:]    =", a[3:])
print("a[-1]    =", a[-1])
print("I[:3,:3] =", I[:3,:3], sep="\n")

print("\nLinear algebra examples:")
print("100+a =", 100+a)
print("I@a   =", I@a)
print("I*a   =", I*a, sep="\n")

plt.title("Plotting example")
plt.ylabel("i $x^2$")
for i in range(4):
    plt.plot(i * a**2, label="i = %d"%i)
plt.legend(); 

### Data assimilation (DA) is:

 * the calibration of big models with big data;
 * the fusion of forecasts with observations.

This illustrated on the right (source: Data Assimilation Research Team, <a href="http://www.aics.riken.jp">www.aics.riken.jp</a>) as a "bridge" between data and models.
<img align="right" width="400" src="./resources/DA_bridges.jpg" alt='DA "bridges" data and models.'/>

The problem of ***DA*** fits well within the math/stats theory of ***state estimation*** and ***sequential inference***. A concise theoretical overview of DA is given by Wikle and Berliner: [A Bayesian tutorial for data assimilation](http://web-static-aws.seas.harvard.edu/climate/pdf/2007/Wikle_Berliner_InPress.pdf)

Modern DA builds on state estimation techniques such as the ***Kalman filter (KF)***, which is an algorithm that recursively performs a form of least-squares regression. It was developed to steer the Apollo mission rockets to the moon, but also has applications outside of control systems, such as speech recognition, video tracking, and finance. An [introduction by pictures](http://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/) is provided by Tim Babb. An [interactive tutorial](https://github.com/rlabbe/Kalman-and-Bayesian-Filters-in-Python) has been made by Roger Labbe.

When it was first proposed to apply the KF to DA (specifically, weather forecasting), the idea sounded ludicrous because of some severe **technical challenges in DA (vs. "classic" state estimation)**:
 * size of data and models;
 * nonlinearity of models;
 * sparsity and inhomogeneous-ness of data.

Some of these challenges may be recognized in the video below.

In [ ]:
envisat_video()

### The EnKF is:
an ensemble (Monte-Carlo) formulation of the KF
that manages (fairly well) to deal with the above challenges in DA.

For those familiar with the method of 4D-Var, **further advantages of the EnKF** include it being:
 * Non-invasive: the models are treated as black boxes, and no explicit jacobian is required.
 * Bayesian: 
   * provides ensemble of possible realities;
       - arguably the most practical form of "uncertainty quanitification";
       - ideal way to initialize "ensemble forecasts";
   * uses "flow-dependent" background covariances in the analysis.
 * Embarrasingly parallellizable:
   * distributed accross realizations for model forecasting;
   * distributed accross local domains for observation analysis.
   
The rest of this tutorial provides an EnKF-centric presentation of DA; it also has a [theoretical companion](./resources/companion/DA_tut.pdf).

### DAPPER example
This tutorial builds on the underlying package, DAPPER, made for academic reserach in DA and its dissemination. For example, the code below is taken from  `DAPPER/example_1.py`. It illustrates DA on a small toy problem.

Run the cells in order and try to interpret the output.

<mark><font size="-1">
<em>Don't worry</em> if you can't understand what's going on -- we will discuss it later throughout the tutorials. 
</font></mark>


In [ ]:
from dapper.mods.Lorenz63.sak12 import HMM
HMM.t.T = 30
# print(HMM)
xx,yy = simulate(HMM)

In [ ]:
config = EnKF_N(N=4, store_u=True)
stats = config.assimilate(HMM,xx,yy)

In [ ]:
avrgs = stats.average_in_time()
# print(avrgs)
print_averages(config,avrgs,[],['rmse_a','rmv_a'])

In [ ]:
replay(stats,figlist="all")

### Exercises

<mark><font size="-1">
    Exercises marked with an asterisk (*) are <em>optional.</em>
</font></mark>

**Exc 1.2:** Word association.
Fill in the `X`'s in the table to group the words according to meaning.

`Filtering, Sample, Random, Measurements, Kalman filter (KF), Monte-Carlo, Observations, Set of draws, State estimation, Data fusion`

```
Data Assimilation (DA)     Ensemble      Stochastic     Data        
------------------------------------------------------------
X                          X             X              X           
X                          X             X              X           
X                          
X
```

In [ ]:
#show_answer('thesaurus 1')

* "The answer" is given from the perspective of DA. Do you agree with it?
* Can you describe the (important!) nuances between the similar words?

**Exc 1.3*:** Word association (advanced).
Group these words:

`Inverse problems, Sample point, Probability, Particle, Sequential, Inversion, Realization, Relative frequency, Iterative, Estimation, Single draw, Serial, Approximation, Regression, Fitting`


    Statistical inference    Ensemble member     Quantitative belief    Recursive 
    -----------------------------------------------------------------------------
    X                        X                   X                      X         
    X                        X                   X                      X         
    X                        X                                          X         
    X                        X
    X                        
    X                        

In [ ]:
#show_answer('thesaurus 2')

**Exc 1.5*:** Prepare to discuss the following questions. Use any tool at your disposal.
* (a) What is DA?
* (b) What are "state variables"? How do they differ from parameters?
* (c) What is a "dynamical system"?
* (d) Is DA a science, an engineering art, or a dark art?
* (e) What is the point of "Hidden Markov Models"?

In [ ]:
#show_answer('Discussion topics 1')

### Next: [Bayesian inference](T2%20-%20Bayesian%20inference.ipynb)